In [134]:
import pandas_datareader.data as reader
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
import yfinance as yf

In [ ]:
#Extracting data from yahoo finance

In [149]:
ticker = 'GOOG'
print(ticker)
a1 = yf.Ticker(ticker)
df1 = a1.history(ticker , start="2015-01-01", end="2020-01-01")
tickerret = df1['Close'].pct_change()
tickerret

GOOG


Date
2014-12-31         NaN
2015-01-02   -0.003020
2015-01-05   -0.020846
2015-01-06   -0.023177
2015-01-07   -0.001713
                ...   
2019-12-24   -0.003914
2019-12-26    0.012534
2019-12-27   -0.006256
2019-12-30   -0.011650
2019-12-31    0.000659
Name: Close, Length: 1259, dtype: float64

In [ ]:
#Calculating monthly returns

In [136]:
tickerret_mnth = tickerret.resample('M').agg(lambda x: (x+1).prod() - 1)
tickerret_mnth = tickerret_mnth[1:]
tickerret_mnth

Date
2015-01-31    0.015426
2015-02-28    0.044676
2015-03-31   -0.018625
2015-04-30   -0.016760
2015-05-31   -0.009733
2015-06-30   -0.021800
2015-07-31    0.201917
2015-08-31   -0.011764
2015-09-30   -0.015900
2015-10-31    0.168288
2015-11-30    0.044724
2015-12-31    0.021923
2016-01-31   -0.020991
2016-02-29   -0.060812
2016-03-31    0.067615
2016-04-30   -0.069723
2016-05-31    0.061630
2016-06-30   -0.059289
2016-07-31    0.110808
2016-08-31   -0.002263
2016-09-30    0.013350
2016-10-31    0.009327
2016-11-30   -0.033778
2016-12-31    0.018178
2017-01-31    0.032352
2017-02-28    0.033158
2017-03-31    0.007714
2017-04-30    0.092097
2017-05-31    0.065014
2017-06-30   -0.058174
2017-07-31    0.023957
2017-08-31    0.009490
2017-09-30    0.021058
2017-10-31    0.059983
2017-11-30    0.004692
2017-12-31    0.024466
2018-01-31    0.118062
2018-02-28   -0.055738
2018-03-31   -0.066025
2018-04-30   -0.014015
2018-05-31    0.066507
2018-06-30    0.028258
2018-07-31    0.091077
2018-0

In [ ]:
#Extracting Fama-French data

In [137]:
factors_data = reader.DataReader('F-F_Research_Data_Factors','famafrench',"2015-01-01","2020-01-01")[0]
factors_data = factors_data[1:]

In [138]:
tickerret_mnth.index = factors_data.index

In [139]:
tickerret_mnth

Date
2015-02    0.015426
2015-03    0.044676
2015-04   -0.018625
2015-05   -0.016760
2015-06   -0.009733
2015-07   -0.021800
2015-08    0.201917
2015-09   -0.011764
2015-10   -0.015900
2015-11    0.168288
2015-12    0.044724
2016-01    0.021923
2016-02   -0.020991
2016-03   -0.060812
2016-04    0.067615
2016-05   -0.069723
2016-06    0.061630
2016-07   -0.059289
2016-08    0.110808
2016-09   -0.002263
2016-10    0.013350
2016-11    0.009327
2016-12   -0.033778
2017-01    0.018178
2017-02    0.032352
2017-03    0.033158
2017-04    0.007714
2017-05    0.092097
2017-06    0.065014
2017-07   -0.058174
2017-08    0.023957
2017-09    0.009490
2017-10    0.021058
2017-11    0.059983
2017-12    0.004692
2018-01    0.024466
2018-02    0.118062
2018-03   -0.055738
2018-04   -0.066025
2018-05   -0.014015
2018-06    0.066507
2018-07    0.028258
2018-08    0.091077
2018-09    0.000764
2018-10   -0.020292
2018-11   -0.097782
2018-12    0.016401
2019-01   -0.053745
2019-02    0.077983
2019-03    0.00

In [ ]:
#Merging both dataframes

In [140]:
merged = pd.merge(tickerret_mnth,factors_data,on='Date')

In [144]:
merged[['Mkt-RF','SMB','HML','RF']] = merged[['Mkt-RF','SMB','HML','RF']]/100
merged['Close - RF'] = merged.Close - merged.RF
merged

,Close,Mkt-RF,SMB,HML,RF,Close - RF
Date,,,,,,
2015-02,0.015426,0.0614,0.0052,-0.0181,0.0000,0.015426
2015-03,0.044676,-0.0112,0.0302,-0.0041,0.0000,0.044676
2015-04,-0.018625,0.0059,-0.0304,0.0188,0.0000,-0.018625
2015-05,-0.016760,0.0136,0.0089,-0.0110,0.0000,-0.016760
2015-06,-0.009733,-0.0153,0.0285,-0.0074,0.0000,-0.009733
2015-07,-0.021800,0.0154,-0.0409,-0.0421,0.0000,-0.021800
2015-08,0.201917,-0.0604,0.0044,0.0273,0.0000,0.201917
2015-09,-0.011764,-0.0307,-0.0261,0.0039,0.0000,-0.011764
2015-10,-0.015900,0.0775,-0.0198,-0.0025,0.0000,-0.015900


In [145]:
y = merged['Close - RF']
x = merged[['Mkt-RF','SMB','HML']]
x_const = sm.add_constant(x)

In [ ]:
#Running Regression

In [146]:
model = sm.OLS(y,x_const)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Close - RF   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     2.062
Date:                Sat, 24 Jul 2021   Prob (F-statistic):              0.116
Time:                        20:44:40   Log-Likelihood:                 89.285
No. Observations:                  60   AIC:                            -170.6
Df Residuals:                      56   BIC:                            -162.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0209      0.008      2.721      0.009       0.006       0.036
Mkt-RF        -0.5080      0.216     -2.352      0.022      -0.941      -0.075
SMB            0.3496      0.321      1.089      0.281      -0.294       0.993
HML           -0.2375      0.277     -0.857      0.395      -0.793       0.318
==============================================================================
Omnibus:                        2.633   Durbin-Watson:                   2.072
Prob(Omnibus):                  0.268   Jarque-Bera (JB):                1.973
Skew:                           0.434   Prob(JB):                        0.373
Kurtosis:                       3.186   Cond. No.                         46.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""